In [ ]:
%pip install -r requirements.txt

In [1]:
import olivia_finder.scraping as scraping
from olivia_finder.package_manager import PackageManager

In [2]:
data_source = scraping.BiocScraper(use_logger=False)

In [4]:
package_list = data_source.obtain_package_names()
package_list[:10]

['BiocGenerics',
 'S4Vectors',
 'BiocVersion',
 'GenomeInfoDb',
 'IRanges',
 'Biobase',
 'zlibbioc',
 'XVector',
 'Biostrings',
 'BiocParallel']

In [5]:
bioconductor = PackageManager(data_source)

In [6]:
p = bioconductor.obtain_package("DeepBlueR")
p.print()

Package:
  name: DeepBlueR
  version: 1.24.1
  url: https://www.bioconductor.org/packages/release/bioc/html/DeepBlueR.html
  dependencies:
    GenomicRanges:
    GenomeInfoDb:
    utils:
    rtracklayer:
    diffr:
    settings:
    stringr:
    R:>= 3.3
    data.table:
    XML:
    R.utils:
    RCurl:
    rjson:
    methods:
    dplyr:
    withr:
    filehash:
    foreach:


In [9]:
bioconductor_packages = bioconductor.obtain_packages(
    package_list[:2],
    extend_repo=True, 
    show_progress=True
)
bioconductor_packages

100%|██████████| 2/2 [00:05<00:00,  2.58s/it]


In [10]:
for p in bioconductor_packages:
    p.print()

Package:
  name: BiocGenerics
  version: 0.44.0
  url: https://www.bioconductor.org/packages/release/bioc/html/BiocGenerics.html
  dependencies:
    utils:
    R:>= 4.0.0
    stats:
    methods:
    graphics:
Package:
  name: S4Vectors
  version: 0.36.2
  url: https://www.bioconductor.org/packages/release/bioc/html/S4Vectors.html
  dependencies:
    utils:
    R:>= 4.0.0
    stats:
    stats4:
    methods:
    BiocGenerics:


In [11]:
bioconductor_packages = bioconductor.obtain_packages(extend_repo=True, show_progress=True)

100%|██████████| 2183/2183 [04:00<00:00,  9.07it/s]


In [14]:
import pickle
# Save the bioconductor package manager as a pickle file
with open("./results/package_managers/bioconductor_pm_scraping.pkl", "wb") as f:
    pickle.dump(bioconductor_packages, f)

In [16]:
# Store the package manager as a adjacency list
b_df = bioconductor.to_full_adj_list()

# Store the package manager as a adjacency list
b_df.to_csv("./results/csv_datasets/bioconductor_adjlist_scraping.csv", index=False)
